In [27]:
import os
import openai
import sys
import numpy as np

In [28]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY_']
HUGGINGFACE_TOKEN = os.environ['HUGGINGFACE_TOKEN']


In [29]:
from sentence_transformers import SentenceTransformer

model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_function = SentenceTransformer(model_name)

In [30]:
from langchain.vectorstores import Chroma
persist_directory = 'db/chroma'

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)

## Document Loading and Splitting

In [31]:
from langchain.document_loaders import PyPDFLoader

In [32]:
loader = PyPDFLoader('data/ReAct.pdf')
pages = loader.load()

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 100
)

In [34]:
splits = splitter.split_documents(pages)
len(splits)

73

In [35]:
splits[0].page_content

'Published as a conference paper at ICLR 2023\nREAC T: S YNERGIZING REASONING AND ACTING IN\nLANGUAGE MODELS\nShunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2\n1Department of Computer Science, Princeton University\n2Google Research, Brain team\n1{shunyuy,karthikn}@princeton.edu\n2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com\nABSTRACT\nWhile large language models (LLMs) have demonstrated impressive performance\nacross tasks in language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action\nplan generation) have primarily been studied as separate topics. In this paper, we\nexplore the use of LLMs to generate both reasoning traces and task-speciﬁc actions\nin an interleaved manner, allowing for greater synergy between the two: reasoning\ntraces help the model induce, track, and update action plans as well as handle\nexceptions, while actions allow it to 

In [36]:
temp = embedding_function.encode(splits[0].page_content)
print(temp)

[-1.42610213e-02 -1.00450464e-01  5.63753508e-02 -4.76643853e-02
  3.94125842e-02  6.60193060e-03  6.64848685e-02  5.35975285e-02
  9.59211141e-02  3.18779238e-02 -3.51582319e-02 -3.34922858e-02
  5.00640534e-02  2.25127488e-02  1.10441603e-01 -1.65781528e-02
  1.40233627e-02 -3.54926032e-03 -1.04830660e-01 -7.76339397e-02
  6.04142994e-02 -2.94648502e-02  3.89858894e-02 -3.81193310e-02
 -1.00444049e-01 -2.74950191e-02  2.60499828e-02 -7.76545629e-02
  5.04011922e-02  1.95956845e-02  6.54101297e-02  4.33976427e-02
  1.21673066e-02  1.39522394e-02 -8.21192637e-02  1.36251256e-01
 -5.80241233e-02  2.52280006e-04 -1.92695986e-02 -9.64747369e-02
 -8.16828236e-02 -3.09433434e-02  5.04717380e-02 -5.84152602e-02
  9.27214026e-02 -5.27057275e-02 -3.17007080e-02 -8.79268919e-04
 -7.07282647e-02  3.39548034e-03 -1.18072145e-01 -1.58097241e-02
  1.02372235e-02 -1.07866814e-02  8.86400230e-03  5.31580951e-03
  2.22819168e-02  4.35798541e-02  2.89247930e-02 -8.27872008e-02
 -9.46392938e-02 -9.24950

## Create Embeddings and store them in DB

In [48]:
embeddings = []
documents = []
metadata = []
ids = []
count = 0
# .encode("hello how are you")
for split in splits:
    embeddings.append(list((embedding_function.encode(split.page_content).astype(float))))
    documents.append(split.page_content)
    metadata.append(split.metadata)
    ids.append(f"id{count}")
    count += 1

    

# print(len(embeddings))

In [40]:
print(len(documents))

73


## Creating VectorDB from Chroma

In [41]:
import chromadb
chroma_client = chromadb.Client()

In [ ]:
collection = chroma_client.create_collection(name='ReActPaper')

In [49]:
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadata,
    ids=ids
)

In [67]:
query = "Shuang Li, Xavier Puig, Chris Paxton,"

In [68]:
results = collection.query(
    query_texts=query,
    n_results=3
)

In [73]:
for doc in results['documents']:
    print(doc)
    print(len(results['documents']))
# print(**results['documents'])

['Published as a conference paper at ICLR 2023\nMethod Pick Clean Heat Cool Look Pick 2 All\nAct (best of 6) 88 42 74 67 72 41 45\nReAct (avg) 65 39 83 76 55 24 57\nReAct (best of 6) 92 58 96 86 78 41 71\nReAct-IM (avg) 55 59 60 55 23 24 48\nReAct-IM (best of 6) 62 68 87 57 39 33 53\nBUTLER g(best of 8) 33 26 70 76 17 12 22\nBUTLER (best of 8) 46 39 74 100 22 24 37\nTable 3: AlfWorld task-speciﬁc success rates (%). BUTLER and\nBUTLER gresults are from Table 4 of Shridhar et al. (2020b). All\nmethods use greedy decoding, except that BUTLER uses beam search.Method Score SR\nAct 62.3 30.1\nReAct 66.6 40.0\nIL 59.9 29.1\nIL+RL 62.4 28.7\nHuman82.1 59.6Expert\nTable 4: Score and suc-\ncess rate (SR) on Web-\nshop. IL/IL+RL taken\nfrom Yao et al. (2022).\ntrained with 1,012 human annotated trajectories, and a imitation + reinforcement learning (IL + RL)\nmethod additionally trained with 10,587 training instructions.\nResults ReAct outperforms Act on both ALFWorld (Table 3) and Webshop (Table

## Integrate an LLM via API to test the qualitative performance of the model